## Conv Filter Visualization

Visualization of the filters of VGG16, via gradient ascent in input space. Using tensorflow backend.

Click this link to download the trained VGG16 weights file: [![link](http://saikatbasak.in/public/img/link.gif "Link")](https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5)

In [ ]:
from __future__ import print_function

import os
import time
import numpy as np
from scipy.misc import imsave

from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D
from keras.applications import imagenet_utils
from keras import backend as K

K.set_image_dim_ordering('tf')

In [ ]:
""" Utility function to convert tensors into valid images. """
def deprocess_image(x):
    # normalize tensor with center on 0. and std 0.1
    # note to self: Need to wrap my head around the math
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1
    
    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)
    
    # convert to RGB array
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

""" Utility function to normalize a tensor by its L2 norm """
def normalize(x):
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

In [ ]:
""" Building the VGG16 model without the top-most fully connected layers. """
# determine proper input shape
input_shape = imagenet_utils._obtain_input_shape(None,
                                                 default_size=224,
                                                 min_size=48,
                                                 dim_ordering='tf',
                                                 include_top=False)

img_input = Input(shape=input_shape)

# Block 1
x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv1')(img_input)
x = Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1')(x)
x = Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv1')(x)
x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv2')(x)
x = Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv1')(x)
x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv2')(x)
x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv1')(x)
x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv2')(x)
x = Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

model = Model(img_input, x, name='vgg16')

model.summary()

In [ ]:
""" Load pre-trained weights. """
model.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [ ]:
# dimensions of the generated pictures for each filter.
img_width, img_height = 128, 128

# the name of the layer we want to visualize
layer_names = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv1', 'block5_conv1']

# set placeholder for the image
input_img = model.input

# get symbolic outputs from each "key" layer
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

# visualize layers in listed in layer_names
for layer_names_index in range(0, len(layer_names)):
    kept_filters = []
    
    # scanning the first 64 filters
    for filter_index in range(0, 64):
        print('Processing filter %d' % filter_index)
        start_time = time.time()

        # building a loss function that maximizes the activation
        # of the n-th filter of the layer considered
        layer_output = layer_dict[layer_names[layer_names_index]].output
        loss = K.mean(layer_output[:, :, :, filter_index])

        # compute gradient of input image wrt the computed loss
        grads = K.gradients(loss, input_img)[0]

        # normalize the gradient
        grads = normalize(grads)

        # this function returns the loss and grads given the input image
        iterate = K.function([input_img], [loss, grads])

        # step size for gradient ascent
        step = 1.

        # creating a gray image with random noise
        input_img_data = np.random.random((1, img_width, img_height, 3))
        input_img_data = (input_img_data - 0.5) * 20 + 128

        # running gradient ascent for 20 steps
        for i in range(20):
            loss_value, grads_value = iterate([input_img_data])
            input_img_data += grads_value * step

            print('Current loss value:', loss_value)
            if loss_value <= 0.:
                # skip filters that are stuck at 0
                break

        # decode the resulting input image
        if loss_value > 0:
            img = deprocess_image(input_img_data[0])
            kept_filters.append((img, loss_value))

        end_time = time.time()
        print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

    # stiching the best 16 filters on a 4 x 4 grid.
    n = 4

    # filters that have the highest loss are assumed to be better-looking
    kept_filters.sort(key=lambda x: x[1], reverse=True)
    kept_filters = kept_filters[:n * n]

    # building a black canvas with 8px margin
    margin = 8
    width = n * img_width + (n-1) * margin
    height = n * img_height + (n-1) * margin
    stitched_filters = np.zeros((width, height, 3))

    # fill the canvas with saved filters
    for i in range(n):
        for j in range(n):
            img, loss = kept_filters[i*n + j]
            stitched_filters[(img_width+margin)*i: (img_width+margin)*i+img_width,
                             (img_height+margin)*j: (img_height+margin)*j+img_height, :] = img

    # create sub-dir and save the result to disk
    if not os.path.exists('images'):
        os.makedirs('images')
    
    imsave('images/stitched_filters_%dx%d_%s.png' %
           (n, n, layer_names[layer_names_index]), stitched_filters)